In [3]:
!pip install requests

In [2]:
API_KEY = 'SdgqgTuw50TvJ3WwUfovjopdhDmaPlwTVpCGOIMZ'

In [4]:
import requests

from datetime import datetime


In [6]:
asteroids_data = []
target = 10000

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

response = requests.get(url)
data = response.json()
details = data['near_earth_objects']
while len(asteroids_data) < target:
      
      for date,ast_details in details.items():
          for ast in ast_details:
            asteroids_data.append(dict(
                ast_id = int(ast['id']),
                neo_reference_id = int(ast['neo_reference_id']),
                name = ast['name'],
                absolute_magnitude_h = float(ast['absolute_magnitude_h']),
                estimated_diameter_min_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                estimated_diameter_max_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                is_potentially_hazardous_asteroid = ast['is_potentially_hazardous_asteroid'],
                close_approach_date = datetime.strptime(ast['close_approach_data'][0]['close_approach_date'],"%Y-%m-%d"	),
                relative_velocity_kmph = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                astronomical = float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
                miss_distance_km = float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
                miss_distance_lunar = float(ast['close_approach_data'][0]['miss_distance']['lunar']),
                orbiting_body = ast['close_approach_data'][0]['orbiting_body']

            ))

            if len(asteroids_data) == target:
                break
          if len(asteroids_data) == target:
                break
      url = data['links']['next']





In [7]:
len(asteroids_data)

10000

In [11]:
type(asteroids_data[0]['close_approach_date'])

datetime.datetime

In [13]:
asteroids_data

[{'ast_id': 2415949,
  'neo_reference_id': 2415949,
  'name': '415949 (2001 XY10)',
  'absolute_magnitude_h': 19.37,
  'estimated_diameter_min_km': 0.3552670883,
  'estimated_diameter_max_km': 0.7944013596,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity_kmph': 57205.8951204341,
  'astronomical': 0.3372535274,
  'miss_distance_km': 50452409.349026635,
  'miss_distance_lunar': 131.1916221586,
  'orbiting_body': 'Earth'},
 {'ast_id': 3160747,
  'neo_reference_id': 3160747,
  'name': '(2003 SR84)',
  'absolute_magnitude_h': 26.0,
  'estimated_diameter_min_km': 0.0167708462,
  'estimated_diameter_max_km': 0.0375007522,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity_kmph': 38589.054833182,
  'astronomical': 0.1323425924,
  'miss_distance_km': 19798169.933318187,
  'miss_distance_lunar': 51.4812684436,
  'orbiting_body': 'Earth'},


In [15]:
import pymysql
import pandas as pd
import numpy as np

In [20]:
myconnection = pymysql.connect(host='127.0.0.1', user = 'root', password = 'rootroot')

In [22]:
cursor = myconnection.cursor()

In [43]:
cursor.execute("CREATE DATABASE nasa_db")

1

In [45]:
myconnection.commit()
print("Database nasa_db is created")

Database nasa_db is created


In [17]:
myconnection = pymysql.connect(host='127.0.0.1', user = 'root', password = 'rootroot', database = 'nasa_db')

In [20]:
cursor = myconnection.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE asteroids (
    id INT,
    name VARCHAR(40),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

In [62]:
cursor.execute("""
CREATE TABLE close_approach (
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(40)
)
""")

0

In [66]:
insert_asteroid_query = """
INSERT INTO asteroids (
    id, name, absolute_magnitude_h, estimated_diameter_min_km,
    estimated_diameter_max_km, is_potentially_hazardous_asteroid
) VALUES (%s, %s, %s, %s, %s, %s)
"""

In [68]:
insert_close_approach_query = """
INSERT INTO close_approach (
    neo_reference_id, close_approach_date, relative_velocity_kmph,
    astronomical, miss_distance_km, miss_distance_lunar, orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [70]:
for asteroid in asteroids_data:
    asteroid_values = (
        asteroid['ast_id'],
        asteroid['name'],
        asteroid['absolute_magnitude_h'],
        asteroid['estimated_diameter_min_km'],
        asteroid['estimated_diameter_max_km'],
        asteroid['is_potentially_hazardous_asteroid']
    )
    cursor.execute(insert_asteroid_query, asteroid_values)

In [76]:
for asteroid in asteroids_data:
    close_approach_values = (
            asteroid['neo_reference_id'],
            asteroid['close_approach_date'].strftime("%Y-%m-%d"),
            asteroid['relative_velocity_kmph'],
            asteroid['astronomical'],
            asteroid['miss_distance_km'],
            asteroid['miss_distance_lunar'],
            asteroid['orbiting_body']
        )
    cursor.execute(insert_close_approach_query, close_approach_values)

In [78]:
myconnection.commit()

In [168]:
import warnings
warnings.simplefilter("ignore")

In [170]:
pd.read_sql_query("Select * from ASTEROIDS", myconnection).count()

id                                   10000
name                                 10000
absolute_magnitude_h                 10000
estimated_diameter_min_km            10000
estimated_diameter_max_km            10000
is_potentially_hazardous_asteroid    10000
dtype: int64

In [31]:
pd.read_sql_query("Select * from close_approach", myconnection).count()

/var/folders/ck/v5xhz32d0gj89n964q9c5f_h0000gn/T/ipykernel_18196/2675272023.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("Select * from close_approach", myconnection).count()


neo_reference_id          10000
close_approach_date       10000
relative_velocity_kmph    10000
astronomical              10000
miss_distance_km          10000
miss_distance_lunar       10000
orbiting_body             10000
dtype: int64

In [174]:
pd.read_sql_query("""SELECT asteroids.name, AVG(close_approach.relative_velocity_kmph) AS average_velocity_kmph
FROM
    asteroids
JOIN
    close_approach ON asteroids.id = close_approach.neo_reference_id
GROUP BY
    asteroids.id, asteroids.name
ORDER BY
    asteroids.name ASC""", myconnection)

,name,average_velocity_kmph
0,(2002 AO11),66352.992188
1,(2003 JV14),49177.167969
2,(2003 SR84),38589.054688
3,(2005 YQ96),56413.015625
4,(2007 SG11),21130.544922
...,...,...
106,591179 (2013 EU9),23244.705078
107,613286 (2005 YQ96),56413.042969
108,668499 (2012 AB11),66101.007812
109,669051 (2012 SD22),39682.058594


In [176]:
# 1. Count how many times each asteroid has approached Earth


pd.read_sql_query("""
SELECT asteroids.name AS asteroid_name, COUNT(close_approach.neo_reference_id) AS approach_count from asteroids 
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
GROUP BY asteroids.id, asteroids.name
ORDER BY approach_count DESC, asteroids.name ASC;
  """, myconnection)




,asteroid_name,approach_count
0,(2003 SR84),8281
1,(2005 YQ96),8281
2,(2009 HC21),8281
3,(2010 XA11),8281
4,(2011 YP10),8281
...,...,...
106,585310 (2017 YZ1),8100
107,591179 (2013 EU9),8100
108,668499 (2012 AB11),8100
109,669051 (2012 SD22),8100


In [178]:
            # 2. Average velocity of each asteroid over multiple approaches
            
            pd.read_sql_query("""SELECT asteroids.name, AVG(close_approach.relative_velocity_kmph) AS average_velocity_kmph
            FROM   asteroids
            JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
            GROUP BY asteroids.id, asteroids.name
            ORDER BY asteroids.name ASC""", myconnection)


,name,average_velocity_kmph
0,(2002 AO11),66352.992188
1,(2003 JV14),49177.167969
2,(2003 SR84),38589.054688
3,(2005 YQ96),56413.015625
4,(2007 SG11),21130.544922
...,...,...
106,591179 (2013 EU9),23244.705078
107,613286 (2005 YQ96),56413.042969
108,668499 (2012 AB11),66101.007812
109,669051 (2012 SD22),39682.058594


In [180]:
# 3. List top 10 fastest asteroids

pd.read_sql_query("""

select asteroids.name as NAME , close_approach.relative_velocity_kmph from asteroids JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
 ORDER BY close_approach.relative_velocity_kmph DESC
 LIMIT 10;

""", myconnection)


,NAME,relative_velocity_kmph
0,(2013 NT11),136268.0
1,(2013 NT11),136268.0
2,(2013 NT11),136268.0
3,(2013 NT11),136268.0
4,(2013 NT11),136268.0
5,(2013 NT11),136268.0
6,(2013 NT11),136268.0
7,(2013 NT11),136268.0
8,(2013 NT11),136268.0
9,(2013 NT11),136268.0


In [186]:
# 4. Find potentially hazardous asteroids that have approached Earth more than 3 times

pd.read_sql_query("""
SELECT asteroids.name, COUNT(close_approach.neo_reference_id) AS approach_count
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
WHERE asteroids.is_potentially_hazardous_asteroid = TRUE
GROUP BY asteroids.id, asteroids.name
HAVING COUNT(close_approach.neo_reference_id) > 3
ORDER BY approach_count DESC
""", myconnection)


,name,approach_count
0,(2005 YQ96),8281
1,(2012 SD22),8281
2,613286 (2005 YQ96),8281
3,669051 (2012 SD22),8100
4,450293 (2004 LV3),8100
5,(2008 WZ94),8100
6,(2017 YZ1),8100
7,585310 (2017 YZ1),8100
8,199003 (2005 WJ56),8100
9,434188 (2003 AD23),8100


In [194]:
# 5. Find the month with the most asteroid approaches

pd.read_sql_query("""
SELECT MONTH(close_approach_date) AS month_number, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month_number
ORDER BY approach_count DESC
LIMIT 1;
""", myconnection)

,month_number,approach_count
0,1,10000


In [38]:
# 6.Get the asteroid with the fastest ever approach speed

pd.read_sql_query("""
SELECT asteroids.name, close_approach.relative_velocity_kmph
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
ORDER BY close_approach.relative_velocity_kmph DESC
LIMIT 1;
""", myconnection)


/var/folders/ck/v5xhz32d0gj89n964q9c5f_h0000gn/T/ipykernel_18196/296082290.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("""


,name,relative_velocity_kmph
0,(2013 NT11),136268.0


In [200]:
# 7. Sort asteroids by maximum estimated diameter (descending)

pd.read_sql_query("""
SELECT asteroids.name, asteroids.estimated_diameter_max_km FROM asteroids
GROUP BY asteroids.name, asteroids.estimated_diameter_max_km
ORDER BY asteroids.estimated_diameter_max_km DESC;

""", myconnection)

,name,estimated_diameter_max_km
0,199003 (2005 WJ56),1.386880
1,450293 (2004 LV3),1.018690
2,434188 (2003 AD23),0.903733
3,304640 (2006 WW1),0.871044
4,415949 (2001 XY10),0.794401
...,...,...
106,(2021 UO1),0.014724
107,(2024 AD),0.010521
108,(2022 JM),0.010329
109,(2023 MB3),0.005943


In [226]:
# 8. An asteroid whose closest approach is getting nearer over time(Hint: Use ORDER BY close_approach_date and look at miss_distance). 

pd.read_sql_query("""

SELECT asteroids.name, COUNT(close_approach.neo_reference_id) AS approach_count,close_approach.close_approach_date ,close_approach.miss_distance_lunar	
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
GROUP BY asteroids.id, asteroids.name, close_approach.close_approach_date, close_approach.miss_distance_lunar	
ORDER BY close_approach.miss_distance_lunar ASC
LIMIT 1
""", myconnection)

,name,approach_count,close_approach_date,miss_distance_lunar
0,(2024 AD),8100,2024-01-04,0.631031


In [204]:
# 9. Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.

pd.read_sql_query("""
SELECT asteroids.name, close_approach.close_approach_date, close_approach.miss_distance_km
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
GROUP BY asteroids.name, close_approach.close_approach_date, close_approach.miss_distance_km
ORDER BY close_approach.miss_distance_km ASC;

""", myconnection)

,name,close_approach_date,miss_distance_km
0,(2024 AD),2024-01-04,242676.0
1,(2024 AA),2024-01-02,610572.0
2,(2024 AB2),2024-01-04,744110.0
3,(2024 AM),2024-01-07,1209360.0
4,(2024 AV2),2024-01-01,1452210.0
...,...,...,...
106,(2008 WZ94),2024-01-03,72128900.0
107,585310 (2017 YZ1),2024-01-06,72151900.0
108,(2017 YZ1),2024-01-06,72151900.0
109,(2012 LJ2),2024-01-03,72656400.0


In [206]:
# 10. List names of asteroids that approached Earth with velocity > 50,000 km/h

pd.read_sql_query("""
SELECT DISTINCT asteroids.name
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
WHERE close_approach.relative_velocity_kmph > 50000

""", myconnection)

,name
0,415949 (2001 XY10)
1,(2005 YQ96)
2,(2017 YD8)
3,(2019 KK5)
4,613286 (2005 YQ96)
5,(2024 AA)
6,(2024 AQ1)
7,(2024 AA2)
8,(2015 OD22)
9,(2018 BA)


In [208]:
# 11. Count how many approaches happened per month

pd.read_sql_query("""
SELECT MONTH(close_approach_date) AS month_number, COUNT(neo_reference_id) AS approach_count
FROM close_approach
GROUP BY month_number;
""", myconnection)

,month_number,approach_count
0,1,10000


In [210]:
# 12. Find asteroid with the highest brightness (lowest magnitude value)

pd.read_sql_query("""

SELECT asteroids.name, asteroids.absolute_magnitude_h
FROM asteroids
ORDER BY asteroids.absolute_magnitude_h ASC
LIMIT 1;

""", myconnection)

,name,absolute_magnitude_h
0,199003 (2005 WJ56),18.16


In [214]:
# 13. Get number of hazardous vs non-hazardous asteroids

pd.read_sql_query("""
SELECT asteroids.is_potentially_hazardous_asteroid, COUNT(*) AS asteroid_count
FROM  asteroids
GROUP BY asteroids.is_potentially_hazardous_asteroid
""", myconnection)

,is_potentially_hazardous_asteroid,asteroid_count
0,0,9007
1,1,993


In [224]:
# 14. Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.

pd.read_sql_query("""
SELECT asteroids.name, close_approach.close_approach_date, close_approach.miss_distance_lunar
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
WHERE close_approach.miss_distance_lunar < 1 
ORDER BY close_approach.miss_distance_lunar ASC, asteroids.name ASC
""", myconnection)

,name,close_approach_date,miss_distance_lunar
0,(2024 AD),2024-01-04,0.631031
1,(2024 AD),2024-01-04,0.631031
2,(2024 AD),2024-01-04,0.631031
3,(2024 AD),2024-01-04,0.631031
4,(2024 AD),2024-01-04,0.631031
...,...,...,...
8095,(2024 AD),2024-01-04,0.631031
8096,(2024 AD),2024-01-04,0.631031
8097,(2024 AD),2024-01-04,0.631031
8098,(2024 AD),2024-01-04,0.631031


In [220]:
# 15. Find asteroids that came within 0.05 AU(astronomical distance)

pd.read_sql_query("""
SELECT  asteroids.name, close_approach.astronomical
FROM asteroids
JOIN close_approach ON asteroids.id = close_approach.neo_reference_id
WHERE close_approach.astronomical < 0.05
GROUP BY  asteroids.name, close_approach.astronomical
""", myconnection)

,name,astronomical
0,(2019 KK5),0.026097
1,(2023 YR),0.011564
2,(2024 AA),0.004081
3,(2024 AQ1),0.043829
4,(2023 YR1),0.016608
5,(2023 YY1),0.027132
6,(2024 AV2),0.009707
7,(2024 AA6),0.049142
8,(2024 AD),0.001622
9,(2024 AB2),0.004974
